In [ ]:
import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
eagle_df = pd.read_csv('/content/drive/Shareddrives/SMC 2023 Challenge 7/day_outage.csv')
eagle_df = eagle_df[['fips_code', 'sum_mean', 'run_start_int']]
eagle_df

,fips_code,sum_mean,run_start_int
0,1001,3.000000,1414800000
1,1001,42.000000,1414886400
2,1001,4.500000,1414972800
3,1001,2.000000,1415232000
4,1001,2.285714,1415318400
...,...,...,...
5545206,78030,501.648352,1667865600
5545207,78030,82.052083,1667952000
5545208,78030,NaN,1668038400
5545209,78030,NaN,1668124800


In [ ]:
weather_events = gpd.read_file('/content/drive/Shareddrives/SMC 2023 Challenge 7/Severe_Weather_Events/weatherall.shp')
weather_events.head()

,PHENOM,POLY_BEG,POLY_END,geometry
0,MA,201407120123,201407120139,"POLYGON ((-79.58000 24.53000, -79.63000 24.480..."
1,MA,201402122121,201402122145,"POLYGON ((-80.10000 24.92000, -80.00000 24.850..."
2,MA,201407180042,201407180117,"POLYGON ((-79.84000 24.75000, -80.14000 24.900..."
3,MA,201411100737,201411100809,"POLYGON ((-79.68000 24.79000, -79.82000 24.720..."
4,MA,201402122219,201402122246,"POLYGON ((-79.96000 24.77000, -80.08000 24.870..."


In [ ]:
weather_events['ISSUED'] = weather_events[['POLY_BEG']]
weather_events['EXPIRED'] = weather_events[['POLY_END']]
weather_events = weather_events[['PHENOM', 'ISSUED', 'EXPIRED', 'geometry']]
weather_events = weather_events.dropna()
# weather_winter = gpd.read_file('/content/drive/Shareddrives/SMC 2023 Challenge 7/Severe_Weather_Events/weatherall_winter.shp')
# weather_winter = weather_winter[['PHENOM', 'ISSUED', 'geometry']]
# weather_winter = weather_winter.dropna()

In [ ]:
weather_events['ISSUED'] = weather_events['ISSUED'].map(str).str[4:6] + '/' +  weather_events['ISSUED'].map(str).str[6:8] + '/' + weather_events['ISSUED'].map(str).str[:4]
weather_events['EXPIRED'] = weather_events['EXPIRED'].map(str).str[4:6] + '/' +  weather_events['EXPIRED'].map(str).str[6:8] + '/' + weather_events['EXPIRED'].map(str).str[:4]


# weather_winter['ISSUED'] = weather_winter['ISSUED'].map(str).str[4:6] + '/' +  weather_winter['ISSUED'].map(str).str[6:8] + '/' + weather_winter['ISSUED'].map(str).str[:4]
# weather_winter.tail()

In [ ]:
weather_events.tail()

,PHENOM,ISSUED,EXPIRED,geometry
857812,SV,07/11/2022,07/11/2022,"POLYGON ((-148.55000 65.13000, -148.41000 65.0..."
857813,SV,07/11/2022,07/11/2022,"POLYGON ((-147.89000 64.68000, -147.88000 64.4..."
857814,SV,07/06/2022,07/06/2022,"POLYGON ((-147.18000 66.08000, -147.21000 65.9..."
857815,SV,07/12/2022,07/12/2022,"POLYGON ((-144.61000 65.60000, -144.22000 65.3..."
857816,SV,07/10/2022,07/10/2022,"POLYGON ((-144.71000 64.52000, -144.98000 64.1..."


In [ ]:
weather_events['TIME'] = pd.to_datetime(weather_events['ISSUED']).dt.strftime("%Y-%m-%dT%H:%M:%S")
weather_events['ISSUED'] = pd.to_datetime(weather_events['ISSUED']).astype(int) / 10**9
weather_events['EXPIRED'] = pd.to_datetime(weather_events['EXPIRED']).astype(int) / 10**9
# weather_events.tail()
# weather_winter['ISSUED'] = pd.to_datetime(weather_winter['ISSUED']).dt.strftime("%Y-%m-%dT%H:%M:%S")
# weather_winter.tail()

In [ ]:
fname = '/content/drive/Shareddrives/SMC 2023 Challenge 7/cb_2018_us_county_500k/cb_2018_us_county_500k.shp'
counties = gpd.read_file(fname)
counties = counties[['GEOID', 'NAME', 'geometry']]
counties['GEOID'] = counties['GEOID'].astype(int)

In [ ]:
counties = counties.rename(columns={'GEOID': 'fips_code'})

In [ ]:
sorted_df = eagle_df.sort_values(['fips_code',
                     'run_start_int']).reset_index(drop=True)

In [ ]:
eagle_df = eagle_df[eagle_df.sum_mean != 0]
eagle_df = eagle_df.fillna(1)

In [ ]:
joined_df = eagle_df.merge(counties, on='fips_code')

In [ ]:
joined_df['log_sum'] = np.log10(joined_df['sum_mean'])

In [ ]:
joined_df = joined_df[['fips_code', 'run_start_int', 'log_sum', 'geometry']]

In [ ]:
max_color = max(joined_df['log_sum'])
min_color = min(joined_df['log_sum'])
cmap = cm.linear.YlOrRd_09.scale(min_color, max_color)
joined_df['color'] = joined_df['log_sum'].map(cmap)

In [ ]:
county_list = joined_df['fips_code'].unique().tolist()
county_idx = range(len(county_list))

style_dict = {}
for i in county_idx:
    county = county_list[i]
    result = joined_df[joined_df['fips_code'] == county]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['run_start_int']] = {'color': r['color'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

In [ ]:
counties_df = joined_df[['geometry']]
counties_gdf = gpd.GeoDataFrame(counties_df)
counties_gdf = counties_gdf.drop_duplicates().reset_index()

In [ ]:
from folium.plugins.timestamped_geo_json import TimestampedGeoJson
from folium.plugins import TimeSliderChoropleth
import gc

slider_map = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')

_ = TimeSliderChoropleth(
    data=counties_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)

# weatherw_geojson = []
# for _, event in weather_winter.iterrows():
#     timestamp = event['ISSUED']
#     geometry = event.geometry


#     geojson = {
#         'type': 'Feature',
#         'geometry': geometry.__geo_interface__,
#         'properties': {'time': timestamp, 'style': {
#             #'fillColor': None,  # Set 'None' to remove the fill color
#             'color': 'cyan',   # Set the border color
#            'weight': 1,        # Set the border weight
#            'fillOpacity': 0.2
#         }}
#     }

#     weatherw_geojson.append(geojson)

# TimestampedGeoJson(
#     {'type': 'FeatureCollection', 'features': weatherw_geojson},
#     period='P1D',
#     duration='P1D',
#     add_last_point=True,
#     auto_play=False,
#     time_slider_drag_update=True
# ).add_to(slider_map)

weathersv_geojson = []
for _, row in weather_events.iterrows():
    start_time = pd.Timestamp(row['ISSUED'])
    end_time = pd.Timestamp(row['EXPIRED'])
    timestamp = row['TIME']
    geometry = row.geometry

    current_time = start_time
    while current_time <= end_time:
        geojson = {
            'type': 'Feature',
            'geometry': geometry.__geo_interface__,
            'properties': {'time': timestamp, 'style': {
                'fillColor': None,
                'color': 'lime',  # Set the border color
                'weight': 1,      # Set the border weight
                'fillOpacity': 0.2
            }}
        }
        weathersv_geojson.append(geojson)
        current_time += pd.Timedelta(days=1)  # Increment by 1 day for the next timestamp

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': weathersv_geojson},
    period='P1D',
    duration='P1D',
    add_last_point=True,
    auto_play=False,
    time_slider_drag_update=True
).add_to(slider_map)

folium.LayerControl().add_to(slider_map)
cmap.caption = "Log sum of customers without power"
slider_map.save(outfile='AllColorsCont.html')